# Train Multi-GPU single machine

Run on V100 8x in tests.

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re, json
from tensorflow_snowflake_class import SnowflakeDataset

import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import dask

import time

import tensorflow as tf
print(tf.__version__)

/srv/conda/envs/saturn/lib/python3.9/site-packages/snowflake/connector/options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.1), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  warn_incompatible_dep(


2.4.1


In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/home/jovyan/git-repos/tensorflow_ds/data/clothing-dataset-small/train',
  seed=123,
  image_size=(224,224),
  batch_size=32).prefetch(2)


Found 3068 files belonging to 10 classes.


In [7]:
def train_multigpu(train_ds, n_epochs, base_lr, batchsize):
    
    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='tensorflow_rnd')
    config=wbargs

    strategy = tf.distribute.MirroredStrategy()
    print('Number of devices: %d' % strategy.num_replicas_in_sync)  

    with strategy.scope():
        model = tf.keras.applications.ResNet50(
            include_top=True,
            weights=None,
            classes=10)

        optimizer = keras.optimizers.Adam(lr=base_lr)
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #integer labels

    # Data
    #dist_dataset = strategy.experimental_distribute_dataset(train_ds)

    start = time.time()
    
    model.fit(train_ds,
        steps_per_epoch=96,
        epochs=n_epochs,
        callbacks=[WandbCallback()])
    
    end = time.time()-start
    print("model training time", end)
    wandb.log({"training_time":end})

    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras/')

In [10]:
model_params = {'n_epochs': 20, 
                'base_lr': .02,
               'batchsize': 32}

wbargs = {**model_params,
    'classes':10,
    'Notes':"tf_baseline",
    'Tags': ['multi', 'gpu', 'tensorflow'],
    'dataset':"Clothing",
    'architecture':"ResNet"}

In [11]:

tester = train_multigpu(train_ds, **model_params)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Epoch 1/20
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 214 all-reduces with algorithm = nccl, num_packs = 1
96/96 [==============================] - 78s 68ms/step - loss: 8.7910 - accuracy: 0.1972
Epoch 2/20
96/96 [==============================] - 7s 65ms/step - loss: 2.1704 - accuracy: 0.2449
Epoch 3/20
96/96 [==============================] - 7s 66ms/step - loss: 2.1324 - accuracy: 0.2549
Epoch 4/20
96/96 [==============================] - 7s 66ms/step - loss: 2.1052 - accuracy: 0.2619
Epoch 5/20
96/96 [==============================] - 7s 65ms/step - loss: 2.0986 - accuracy: 0.2700
Epoch 6/20
96/96 [==============================] - 7s 65ms/step

epoch,19
loss,0.80408
accuracy,0.71578
_runtime,217
_timestamp,1629221097
_step,20
training_time,211.26409


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
accuracy,▁▁▁▁▂▂▂▃▃▄▄▅▆▆▇▇▇███
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
training_time,▁


INFO:tensorflow:Assets written to: model/keras/assets
